In [ ]:
# Implement CNN for CIFAR-10 dataset using PyTorch

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Transform to normalize the data
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#         transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
# )
# Transform to normalize the data and for data augmentation
transform = transforms.Compose(
    [
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)


# Hyper-parameters
num_epochs = 20
num_classes = 10
batch_size = 32
learning_rate = 0.001

In [ ]:
# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(
    root="./data/", train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data/", train=False, transform=transform
)

# Data loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False
)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [ ]:
# Convolutional neural network with following layers:
# CONV1: with Kernel size (3 ×3), In channels 3, Out channels 32.
# POOL1: Kernel size (2 ×2).
# CONV2: Kernel size (5 ×5), In channels 32, Out channels 64
# POOL2: Kernel size (2 ×2).
# CONV3: Kernel size (3 ×3), In channels 64, Out channels 64.
# FC1: Fully connected layer (also known as Linear layer) with 64 output neurons.
# FC2: Fully connected layer with 10 output neurons.
# Use ReLU as the activation function for all layers apart from the max-pooling layers, and the FC2layer. 
# You need to flatten the CONV3 output before passing as input to FC1. 
# and train for 20 epochs using an Adam optimizer with learning rate 0.001 and batch size 32. 
# Use the categorical cross entropy loss as the loss function

class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv3(x)
        x = F.relu(x)
        # Flatten
        x = x.view(x.size(0), -1)
        # FC layers
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        # Softmax
        x = F.log_softmax(x, dim=1)
        return x
    
model = ConvNet(num_classes).to(device)


In [ ]:
# Now, define the loss function and optimizer. KLdivergence is used. 
criterion = nn.KLDivLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Set the learning rate scheduler
# step_size = 5
# gamma = 0.1
# scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

In [ ]:
# Training and validation loss and accuracy tracking
epoch_train_loss = []
epoch_train_acc = []
epoch_val_loss = []
epoch_val_acc = []

# Now, train the model for 20 epochs
total_step = len(train_loader)
for epoch in range(num_epochs):
    running_train_loss = 0
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()

        # Update the parameters
        optimizer.step()

        # # Update the learning rate using the scheduler
        # scheduler.step()

        running_train_loss += loss.item()

        # Track the accuracy
        _, predicted = torch.max(outputs.data, 1)
        total = labels.size(0)
        correct = (predicted == labels).sum().item()

    epoch_train_loss.append(running_train_loss / len(train_loader))
    epoch_train_acc.append(correct / float(total))

    # Evaluate the model on the validation set
    running_val_loss = 0
    val_correct = 0
    val_total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    epoch_val_loss.append(running_val_loss / len(test_loader))
    epoch_val_acc.append(val_correct / float(val_total))

    # Print the loss for every epoch
    print(
        f"Epoch [{epoch+1}/{num_epochs}, Train Loss: {epoch_train_loss[-1]:.4f}, Train Acc: {epoch_train_acc[-1]:.4f},Val Loss: {epoch_val_loss[-1]:.4f}, Val Acc: {epoch_val_acc[-1]:.4f}"
    )

# Save the model checkpoint
torch.save(model.state_dict(), "model.pth")

In [ ]:
# Plot the loss curve
plt.figure()
plt.plot(range(1, num_epochs + 1), epoch_train_loss, label="Training Loss")
plt.plot(range(1, num_epochs + 1), epoch_val_loss, label="Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Plot the epoch vs. overall validation accuracy and training accuracy
plt.figure()
plt.plot(range(1, num_epochs + 1), epoch_val_acc, label="Validation Accuracy")
plt.plot(range(1, num_epochs + 1), epoch_train_acc, label="Training Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Epoch vs. Accuracy')
plt.legend()
plt.show()

In [ ]:
# Test the model for class-wise accuracy
model.eval()
with torch.no_grad():
    # Class-wise accuracy
    class_correct = [0] * num_classes
    class_total = [0] * num_classes
    for images, labels in test_loader:
        # Run the forward pass
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # Get the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        # Collect the correct predictions for each class
        c = (predicted == labels)
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
    
    # Print the accuracy for each class
    for i in range(num_classes):
        print(f'Accuracy of {classes[i]}: {100 * class_correct[i] / class_total[i]}%')
